In [2]:
import DataProcessing
from pathlib import Path
import numpy as np
from tqdm import tqdm
import pptk
import math


In [3]:
pcd_tup = DataProcessing.load_from_ptx([Path('../../PatrickData/SongoMnara/10.33.47.ptx')])

File: 10.33.47.ptx
Rows: 5056
Cols: 2150
Total points: 10870400



In [4]:
pcd = np.hstack((pcd_tup[0], pcd_tup[1][:, None], pcd_tup[2][:, 1:], pcd_tup[2][:, 0][:, None]))
pcd.shape

(9015171, 7)

In [5]:
# pcd_dtype = [('x', np.float64),('y', np.float64),('z', np.float64),('i', np.float64),('g', np.float64),('b', np.float64),('class', np.float64),]
# pcd.dtype = pcd_dtype

In [6]:
pcd_copy = pcd.copy()

In [7]:
pcd = pcd_copy.copy()

In [11]:
split_indices = np.arange(0, pcd.shape[0], pcd.shape[0] // 10)[1:]

In [1]:
pcd.view('f8,f8,f8,f8,f8,f8,f8').sort(order=['f0'], axis=0)
pcd = np.array_split(pcd, split_indices)

NameError: name 'pcd' is not defined

In [14]:
pcd[0].shape

(901517, 7)

In [15]:

for i, col in enumerate(tqdm(pcd)):
    col = col.view('f8,f8,f8,f8,f8,f8,f8').sort(order=['f1'], axis=1)

100%|██████████| 11/11 [00:00<00:00, 51.99it/s]


In [16]:
pcd = np.vstack(pcd)

grid_mask = np.repeat(np.arange(0, 10, 1), math.ceil(9015171 / 10))[:pcd.shape[0], None]

In [73]:
pcd = np.hstack((pcd, grid_mask))

In [1]:
def test_split_grid(num_points=1000, shape=(3, 3,), value_range=100, visualise_pptk=True):
    """
    Generate a cloud of random points and segment it into a grid.
    :param num_points: number of points to sample
    :param shape: The number of cells in each row/column
    :param value_range: xy value upper bound [0,value_range)
    :param visualise_pptk: visualise the resulting point grid using pptk
    :return:
    """
    points = np.random.randint(0, value_range, (num_points, 3))
    points, grid_mask = split_grid(points, shape)

    if visualise_pptk:
        v = pptk.viewer(points, grid_mask)
        v.set(point_size=.1)
        v.color_map(DataProcessing.turbo_colormap_data)



def split_grid(points, grid_shape):
    """
    Split the points into a grid pattern
    :param points: (n,3) array of points (XYZ)
    :param grid_shape: (x,y,) tuple of grid shape
    :return: modified points array, grid_mask
    """
    # Sort and split array along x-axis
    points.view('i8,i8,i8').sort(order=['f0'], axis=0)

    total_distances = points[:, :2].max(axis=0) - points[:, :2].min(axis=0)
    intervals_x = np.asarray(
        [points[:, 0].min(axis=0) + np.ceil(total_distances[0] / grid_shape[0]) * i for i in range(grid_shape[0])][1:])
    intervals_y = np.asarray(
        [points[:, 1].min(axis=0) + np.ceil(total_distances[1] / grid_shape[1]) * i for i in range(grid_shape[1])][1:])
    interval_idxs_x = [DataProcessing.find_nearest_id(points[:, 0], v) for v in intervals_x]

    points = np.array_split(points, interval_idxs_x)

    # Sort and split resulting columns along y-axis
    for i in tqdm(range(len(points))):
        col = points[i]
        col.view('i8,i8,i8').sort(order=['f1'], axis=0)
        interval_idxs_y = [0] + [DataProcessing.find_nearest_id(col[:, 1], v) for v in intervals_y] + [col.shape[0]]
        col_grid_mask = np.concatenate(
            [np.repeat(i * grid_shape[1] + j, reps - interval_idxs_y[j - 1]) for j, reps in
             enumerate(interval_idxs_y[1:], start=1)])
        points[i] = np.hstack((col, col_grid_mask[:, None]))

    points = np.vstack(points)
    return points[:,:-1], points[:,-1]


In [ ]:
test_split_grid(1024, (3, 3))
